In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 5000

In [4]:
from IPython.display import display, HTML

def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

# Get Data

In [5]:
df_results_orig = pd.read_excel('../data/VAC Data.xlsx', sheet_name=3)
df_mapping_orig = pd.read_excel('../data/VAC Data.xlsx', sheet_name=2)

In [6]:
df_mapping_orig.head()

,Legend SP - Spouse S - Survivor O - Orphan,Statements,CAF - member or Veteran,RCMP - member or Veteran,War Service Veteran - member,Allied Veteran - member,Family Member,still serving,planning on releasing soon,Member in receipt of a VAC disability benefit,served overseas during the Second World War or the Korean War,Member received War Veterans Allowance,has a Disability Pension,received a Disability Award,Disability Pension 1% or greater,service-related illness or injury that will likely make the transition to civilian life difficult,diagnosed medical illness or injury related to service,was medically released or will be medically released,medical release a result of a service related injury or illness,medically release within the last 120 days,released in last 5 years,"needs help with daily activities, such as dressing and personal care","Disability Pension 98% or greater (or any combination of Disability Pension, Disability Award or POW Compensation of 98% or greater)",disability results in need for ongoing care,has a significant health issue that is a result of these conditions,disability benefit results in the need for special clothing or causes excessive wear and tear on your clothing,has previously participated in rehab,currently participating in rehab,Member has successfully completed the program within the last 6 months but has been unable to find gainful employment.,Member no longer participating and has a VAC approved disability benefit considered to be permanent and severe.,"Member has been assessed as having a ""Diminished Earning Capacity.""",member or Veteran of the Regular Force,has a health problem primarily resulting from service which has made the transition from the military to civilian life difficult,requires assistance finding a job transitioning from military to civilian life,lived in Canada prior to enlistment or lived in Canada for a total of 10 years since service,requires financial assistance to meet an emergency or unexpected expense,received Career Impact Allowance,has a permanent and severe impairment related to a VAC approved disability benefit,received Exceptional Incapacity Allowance,"Member has been assessed as having a ""Diminished Earning Capacity."".1",Member has a health condition that prevents him/her from being suitably and gainfully employed,qualifys for the VAC Rehabilitation program because of health condition,Member has a household income less than the CFIS or WVA rates?,Member has released within the last two years,Reserve force Veteran or member completed at least 21 months of full time service during 24 consecutive months or Special Duty Service or emergency service within the last two years,Member receives Canadian Forces Income Supplement,Member Resides in Canada,"Member requires assistance with such things as housekeeping, grounds keeping or daily activities such as dressing and personal care",Without assistance the member would require a nursing home placement or alternative living arrangements,Member is receiving compensation for this illness or injury from the Allied Country,Member resides in a nursing home or care facility,Member has an unpaid caregiver at least 18 years of age,deemed eligible for SISIP-LTD,Member has been assessed as Totally Disabled under SISIP,Has attained the age of 65,"Severe injury after April 1, 2006",At least one of the following apply to the Member's situation as a result of this incident see list 1-8,Member finding it difficult to manage a change or transition in life,You require nursing home type care,You would like to speak with a mental health professional about a personal problem or concern,Spouse or Common-law partner,Separated Spouse or Partner,Survivor,Dependent Child,Orphan (Veteran is deceased),Parent or Extended Family,Caregiver,CAF - Family Member,RCMP - Family Member,War Service Veteran - Family Member,Allied Veteran - Family Member,Member is still serving in the CAF,Family member receiving Survivor's pension,Family member receiving 

In [7]:
df_mapping = df_mapping_orig.rename(columns={df_mapping_orig.columns[0]: "Result ID", 
                                         "Statements": "Scenario",
                                         "Unnamed: 129": "Service Type",
                                         "Unnamed: 127": "Result Name",
                                         "Unnamed: 130": "Applicant",
                                         "Unnamed: 131": "Benefit",
                                         "Unnamed: 132": "Additional Details",
                                         "Unnamed: 128": "Result Description",
                                        })
df_mapping = df_mapping.drop(df_mapping.index[0])

In [8]:
df_mapping.head()

,Result ID,Scenario,CAF - member or Veteran,RCMP - member or Veteran,War Service Veteran - member,Allied Veteran - member,Family Member,still serving,planning on releasing soon,Member in receipt of a VAC disability benefit,served overseas during the Second World War or the Korean War,Member received War Veterans Allowance,has a Disability Pension,received a Disability Award,Disability Pension 1% or greater,service-related illness or injury that will likely make the transition to civilian life difficult,diagnosed medical illness or injury related to service,was medically released or will be medically released,medical release a result of a service related injury or illness,medically release within the last 120 days,released in last 5 years,"needs help with daily activities, such as dressing and personal care","Disability Pension 98% or greater (or any combination of Disability Pension, Disability Award or POW Compensation of 98% or greater)",disability results in need for ongoing care,has a significant health issue that is a result of these conditions,disability benefit results in the need for special clothing or causes excessive wear and tear on your clothing,has previously participated in rehab,currently participating in rehab,Member has successfully completed the program within the last 6 months but has been unable to find gainful employment.,Member no longer participating and has a VAC approved disability benefit considered to be permanent and severe.,"Member has been assessed as having a ""Diminished Earning Capacity.""",member or Veteran of the Regular Force,has a health problem primarily resulting from service which has made the transition from the military to civilian life difficult,requires assistance finding a job transitioning from military to civilian life,lived in Canada prior to enlistment or lived in Canada for a total of 10 years since service,requires financial assistance to meet an emergency or unexpected expense,received Career Impact Allowance,has a permanent and severe impairment related to a VAC approved disability benefit,received Exceptional Incapacity Allowance,"Member has been assessed as having a ""Diminished Earning Capacity."".1",Member has a health condition that prevents him/her from being suitably and gainfully employed,qualifys for the VAC Rehabilitation program because of health condition,Member has a household income less than the CFIS or WVA rates?,Member has released within the last two years,Reserve force Veteran or member completed at least 21 months of full time service during 24 consecutive months or Special Duty Service or emergency service within the last two years,Member receives Canadian Forces Income Supplement,Member Resides in Canada,"Member requires assistance with such things as housekeeping, grounds keeping or daily activities such as dressing and personal care",Without assistance the member would require a nursing home placement or alternative living arrangements,Member is receiving compensation for this illness or injury from the Allied Country,Member resides in a nursing home or care facility,Member has an unpaid caregiver at least 18 years of age,deemed eligible for SISIP-LTD,Member has been assessed as Totally Disabled under SISIP,Has attained the age of 65,"Severe injury after April 1, 2006",At least one of the following apply to the Member's situation as a result of this incident see list 1-8,Member finding it difficult to manage a change or transition in life,You require nursing home type care,You would like to speak with a mental health professional about a personal problem or concern,Spouse or Common-law partner,Separated Spouse or Partner,Survivor,Dependent Child,Orphan (Veteran is deceased),Parent or Extended Family,Caregiver,CAF - Family Member,RCMP - Family Member,War Service Veteran - Family Member,Allied Veteran - Family Member,Member is still serving in the CAF,Family member receiving Survivor's pension,Family member receiving War Veterans Allowance,Family membe

In [22]:
data = df_mapping[['Service Type', "Applicant", "still serving", "Benefit"]].copy()
data = data.drop_duplicates()
data['Benefit'] += ' ; '
data.head()

,Service Type,Applicant,still serving,Benefit
11,CAF,Member,NaN,Disability Benefits ;
31,War Service Veteran,Member,NaN,Disability Benefits ;
32,RCMP,Member,NaN,Disability Benefits ;
33,Allied Veteran,Member,NaN,Disability Benefits ;
61,CAF,Member,NaN,Family Caregiver Relief Benefit ;


## Benefits by Service Type and Applicant

In [23]:
df = data.copy()
df['any serving'] = df['still serving'].isnull()
df['serving'] = (df['still serving'] == 'Y') | df['any serving']
df['released'] = (df['still serving'] == 'N') | df['any serving']
df

,Service Type,Applicant,still serving,Benefit,any serving,serving,released
11,CAF,Member,NaN,Disability Benefits ;,True,True,True
31,War Service Veteran,Member,NaN,Disability Benefits ;,True,True,True
32,RCMP,Member,NaN,Disability Benefits ;,True,True,True
33,Allied Veteran,Member,NaN,Disability Benefits ;,True,True,True
61,CAF,Member,NaN,Family Caregiver Relief Benefit ;,True,True,True
71,CAF,Member,NaN,Attendance Allowance ;,True,True,True
72,War Service Veteran,Member,NaN,Attendance Allowance ;,True,True,True
73,RCMP,Member,NaN,Attendance Allowance ;,True,True,True
74,Allied Veteran,Member,NaN,Attendance Allowance ;,True,True,True
81,CAF,Member,NaN,Clothing Allowance ;,True,True,True


In [24]:
def member_status(row):
    if row['Applicant'] != 'Member':
        return row['Applicant']
    if row['serving'] and row['released']:
        return 'Member-serving-released'
    if row['serving']:
        return 'Member-serving'
    if row['released']:
        return 'Member-released'
    return 'Member'
    
df['Applicant Extended'] = df.apply(member_status, axis=1)
df

,Service Type,Applicant,still serving,Benefit,any serving,serving,released,Applicant Extended
11,CAF,Member,NaN,Disability Benefits ;,True,True,True,Member-serving-released
31,War Service Veteran,Member,NaN,Disability Benefits ;,True,True,True,Member-serving-released
32,RCMP,Member,NaN,Disability Benefits ;,True,True,True,Member-serving-released
33,Allied Veteran,Member,NaN,Disability Benefits ;,True,True,True,Member-serving-released
61,CAF,Member,NaN,Family Caregiver Relief Benefit ;,True,True,True,Member-serving-released
71,CAF,Member,NaN,Attendance Allowance ;,True,True,True,Member-serving-released
72,War Service Veteran,Member,NaN,Attendance Allowance ;,True,True,True,Member-serving-released
73,RCMP,Member,NaN,Attendance Allowance ;,True,True,True,Member-serving-released
74,Allied Veteran,Member,NaN,Attendance Allowance ;,True,True,True,Member-serving-released
81,CAF,Member,NaN,Clothing Allowance ;,True,True,True,Member-serving-released


In [25]:
df2 = pd.pivot_table(df, values='Benefit', index="Service Type", columns=["Applicant Extended"], aggfunc=sum).fillna(' ')
df2

,Applicant Extended,Caregiver,Dependent Child,Member-released,Member-serving,Member-serving-released,Orphan (Veteran is deceased),Parent / Extended Family,Spouse / Common-law Partner,Survivor
Service Type,,,,,,,,,,
Allied Veteran,Benefit,,,,,Disability Benefits ; Attendance Allowance ; Clothing Allowance ; Exceptional Incapacity Allowance ; Veterans Independence Program ; War Veterans Allowance ; Case Management ; Long term care ; VAC Assistance Service ;,,,Disability Benefits ; War Veterans Allowance ; Attendance Allowance ;,Survivor's pension ; Posthumous DP - Spouse ; Veterans Independence Program for Survivors ; Veterans Independence Program for Primary Caregivers ; Funeral and Burial Assistance ;
CAF,Benefit,Memorial Ribbon ; OSISS Family Support ;,OSISS Family Support ;,Exceptional Incapacity Allowance ; Veterans Hiring Act (Priority Hire) ; Rehabilitation Services ; Permanent Impairment Allowance ; Totally and Permanently Incapacitated ; Canadian Forces Income Supplement ; Retirement Income Security Benefit ; Career Transition Services ; Case Management ;,Exceptional Incapacity Allowance ; Veterans Hiring Act (Priority Hire) ; Rehabilitation Services ; Career Transition Services ; Transition Interview ; Case Management ;,Disability Benefits ; Family Caregiver Relief Benefit ; Attendance Allowance ; Clothing Allowance ; Veterans Independence Program ; Critical Injury Benefit ; Long term care ; VAC Assistance Service ;,Posthumous DB - Orphan ; Death Benefit ; Survivor's pension ; Veterans Independence Program for Primary Caregivers ; Funeral and Burial Assistance ; Earnings Loss Benefit ; Canadian Forces Income Supplement ; Education Assistance Plan ; Memorial Ribbon ; OSISS Family Support ;,Memorial Ribbon ;,Family Caregiver Relief Benefit ; Vocational Assistance ; Veteran Family Program ; VAC Assistance Service ; OSISS Family Support ; Pastoral Outreach ;,Survivor's pension ; Veterans Independence Program for Survivors ; Veterans Independence Program for Primary Caregivers ; Funeral and Burial Assistance ; Earnings Loss Benefit (Survivor) ; Supplementary Retirement Benefits ; Retirement Income Security Benefit ; Canadian Forces Income Supplement ; Vocational Assistance ; Education Assistance Plan ; Death Benefit ; Disability Award ; Memorial Ribbon ; OSISS Family Support ;
RCMP,Benefit,,,,,Disability Benefits ; Attendance Allowance ; Clothing Allowance ; Exceptional Incapacity Allowance ; Case Management ; VAC Assistance Service ;,Survivor's pension ;,,,Survivor's pension ; Posthumous DP - Spouse ;
War Service Veteran,Benefit,,,,,Disability Benefits ; Attendance Allowance ; Clothing Allowance ; Exceptional Incapacity Allowance ; Veterans Independence Program ; War Veterans Allowance ; Case Management ; Long term care ; VAC Assistance Service ;,,,Disability Benefits ; War Veterans Allowance ; Attendance Allowance ;,Survivor's pension ; Posthumous DP - Spouse ; Veterans Independence Program for Survivors ; Veterans Independence Program for Primary Caregivers ; Funeral and Burial Assistance ;


In [26]:
df2['Released'] = df2['Member-released'] + df2['Member-serving-released']
df2['Serving'] =  df2['Member-serving'] + df2['Member-serving-released']
df2

,Applicant Extended,Caregiver,Dependent Child,Member-released,Member-serving,Member-serving-released,Orphan (Veteran is deceased),Parent / Extended Family,Spouse / Common-law Partner,Survivor,Released,Serving
Service Type,,,,,,,,,,,,
Allied Veteran,Benefit,,,,,Disability Benefits ; Attendance Allowance ; Clothing Allowance ; Exceptional Incapacity Allowance ; Veterans Independence Program ; War Veterans Allowance ; Case Management ; Long term care ; VAC Assistance Service ;,,,Disability Benefits ; War Veterans Allowance ; Attendance Allowance ;,Survivor's pension ; Posthumous DP - Spouse ; Veterans Independence Program for Survivors ; Veterans Independence Program for Primary Caregivers ; Funeral and Burial Assistance ;,Disability Benefits ; Attendance Allowance ; Clothing Allowance ; Exceptional Incapacity Allowance ; Veterans Independence Program ; War Veterans Allowance ; Case Management ; Long term care ; VAC Assistance Service ;,Disability Benefits ; Attendance Allowance ; Clothing Allowance ; Exceptional Incapacity Allowance ; Veterans Independence Program ; War Veterans Allowance ; Case Management ; Long term care ; VAC Assistance Service ;
CAF,Benefit,Memorial Ribbon ; OSISS Family Support ;,OSISS Family Support ;,Exceptional Incapacity Allowance ; Veterans Hiring Act (Priority Hire) ; Rehabilitation Services ; Permanent Impairment Allowance ; Totally and Permanently Incapacitated ; Canadian Forces Income Supplement ; Retirement Income Security Benefit ; Career Transition Services ; Case Management ;,Exceptional Incapacity Allowance ; Veterans Hiring Act (Priority Hire) ; Rehabilitation Services ; Career Transition Services ; Transition Interview ; Case Management ;,Disability Benefits ; Family Caregiver Relief Benefit ; Attendance Allowance ; Clothing Allowance ; Veterans Independence Program ; Critical Injury Benefit ; Long term care ; VAC Assistance Service ;,Posthumous DB - Orphan ; Death Benefit ; Survivor's pension ; Veterans Independence Program for Primary Caregivers ; Funeral and Burial Assistance ; Earnings Loss Benefit ; Canadian Forces Income Supplement ; Education Assistance Plan ; Memorial Ribbon ; OSISS Family Support ;,Memorial Ribbon ;,Family Caregiver Relief Benefit ; Vocational Assistance ; Veteran Family Program ; VAC Assistance Service ; OSISS Family Support ; Pastoral Outreach ;,Survivor's pension ; Veterans Independence Program for Survivors ; Veterans Independence Program for Primary Caregivers ; Funeral and Burial Assistance ; Earnings Loss Benefit (Survivor) ; Supplementary Retirement Benefits ; Retirement Income Security Benefit ; Canadian Forces Income Supplement ; Vocational Assistance ; Education Assistance Plan ; Death Benefit ; Disability Award ; Memorial Ribbon ; OSISS Family Support ;,Exceptional Incapacity Allowance ; Veterans Hiring Act (Priority Hire) ; Rehabilitation Services ; Permanent Impairment Allowance ; Totally and Permanently Incapacitated ; Canadian Forces Income Supplement ; Retirement Income Security Benefit ; Career Transition Services ; Case Management ; Disability Benefits ; Family Caregiver Relief Benefit ; Attendance Allowance ; Clothing Allowance ; Veterans Independence Program ; Critical Injury Benefit ; Long term care ; VAC Assistance Service ;,Exceptional Incapacity Allowance ; Veterans Hiring Act (Priority Hire) ; Rehabilitation Services ; Career Transition Services ; Transition Interview ; Case Management ; Disability Benefits ; Family Caregiver Relief Benefit ; Attendance Allowance ; Clothing Allowance ; Veterans Independence Program ; Critical Injury Benefit ; Long term care ; VAC Assistance Service ;
RCMP,Benefit,,,,,Disability Benefits ; Attendance Allowance ; Clothing Allowance ; Exceptional Incapacity Allowance ; Case Management ; VAC Assistance Service ;,Survivor's pension ;,,,Survivor's pension ; Posthumous DP - Spouse ;,Disability Benefits ; Attendance Allowance ; Clothing Allowance ; Exceptional Incapacity Allowance ; Case Management ; VAC Assistance

In [28]:
def reformat_benefit_list(bl):
    benefits = sorted(list(set([x.strip() for x in bl.split(';') if x != ' '])))
    return '\n'.join([f"{i+1} - {val}" for i, val in enumerate(benefits)])

df3 = df2.applymap(reformat_benefit_list)

In [31]:
pretty_print(df3.head())

,Applicant Extended,Caregiver,Dependent Child,Member-released,Member-serving,Member-serving-released,Orphan (Veteran is deceased),Parent / Extended Family,Spouse / Common-law Partner,Survivor,Released,Serving
Service Type,,,,,,,,,,,,
Allied Veteran,Benefit,,,,,1 - Attendance Allowance2 - Case Management3 - Clothing Allowance4 - Disability Benefits5 - Exceptional Incapacity Allowance6 - Long term care7 - VAC Assistance Service8 - Veterans Independence Program9 - War Veterans Allowance,,,1 - Attendance Allowance2 - Disability Benefits3 - War Veterans Allowance,1 - Funeral and Burial Assistance2 - Posthumous DP - Spouse3 - Survivor's pension4 - Veterans Independence Program for Primary Caregivers5 - Veterans Independence Program for Survivors,1 - Attendance Allowance2 - Case Management3 - Clothing Allowance4 - Disability Benefits5 - Exceptional Incapacity Allowance6 - Long term care7 - VAC Assistance Service8 - Veterans Independence Program9 - War Veterans Allowance,1 - Attendance Allowance2 - Case Management3 - Clothing Allowance4 - Disability Benefits5 - Exceptional Incapacity Allowance6 - Long term care7 - VAC Assistance Service8 - Veterans Independence Program9 - War Veterans Allowance
CAF,Benefit,1 - Memorial Ribbon2 - OSISS Family Support,1 - OSISS Family Support,1 - Canadian Forces Income Supplement2 - Career Transition Services3 - Case Management4 - Exceptional Incapacity Allowance5 - Permanent Impairment Allowance6 - Rehabilitation Services7 - Retirement Income Security Benefit8 - Totally and Permanently Incapacitated9 - Veterans Hiring Act (Priority Hire),1 - Career Transition Services2 - Case Management3 - Exceptional Incapacity Allowance4 - Rehabilitation Services5 - Transition Interview6 - Veterans Hiring Act (Priority Hire),1 - Attendance Allowance2 - Clothing Allowance3 - Critical Injury Benefit4 - Disability Benefits5 - Family Caregiver Relief Benefit6 - Long term care7 - VAC Assistance Service8 - Veterans Independence Program,1 - Canadian Forces Income Supplement2 - Death Benefit3 - Earnings Loss Benefit4 - Education Assistance Plan5 - Funeral and Burial Assistance6 - Memorial Ribbon7 - OSISS Family Support8 - Posthumous DB - Orphan9 - Survivor's pension10 - Veterans Independence Program for Primary Caregivers,1 - Memorial Ribbon,1 - Family Caregiver Relief Benefit2 - OSISS Family Support3 - Pastoral Outreach4 - VAC Assistance Service5 - Veteran Family Program6 - Vocational Assistance,1 - Canadian Forces Income Supplement2 - Death Benefit3 - Disability Award4 - Earnings Loss Benefit (Survivor)5 - Education Assistance Plan6 - Funeral and Burial Assistance7 - Memorial Ribbon8 - OSISS Family Support9 - Retirement Income Security Benefit10 - Supplementary Retirement Benefits11 - Survivor's pension12 - Veterans Independence Program for Primary Caregivers13 - Veterans Independence Program for Survivors14 - Vocational Assistance,1 - Attendance Allowance2 - Canadian Forces Income Supplement3 - Career Transition Services4 - Case Management5 - Clothing Allowance6 - Critical Injury Benefit7 - Disability Benefits8 - Exceptional Incapacity Allowance9 - Family Caregiver Relief Benefit10 - Long term care11 - Permanent Impairment Allowance12 - Rehabilitation Services13 - Retirement Income Security Benefit14 - Totally and Permanently Incapacitated15 - VAC Assistance Service16 - Veterans Hiring Act (Priority Hire)17 - Veterans Independence Program,1 - Attendance Allowance2 - Career Transition Services3 - Case Management4 - Clothing Allowance5 - Critical Injury Benefit6 - Disability Benefits7 - Exceptional Incapacity Allowance8 - Family Caregiver Relief Benefit9 - Long term care10 - Rehabilitation Services11 - Transition Interview12 - VAC Assistance Service13 - Veterans Hiring Act (Priority Hire)14 - Veterans Independence Program
RCMP,Benefit,,,,,1 - Attendance Allowance2 - Case Management3 - Clothing Allowance4 - Disability Benefits5 - Exceptional Incapacity Allowance6 - VAC Assistance Service,1 - Survivor's pension,,,1 - Posthumo

In [30]:
df3.to_csv('benefits_by_service_applicant.csv')